In [1]:
import praw
import pandas as pd
import datetime
import time
from psaw import PushshiftAPI


# Pushshift API

This API can search for more than 1000 data and specific time range


In [2]:
# Search for more 1000 data

api  = PushshiftAPI()
search_date = datetime.datetime(2020,1,1)
start_epoch = int(search_date.timestamp())
subreddit   = 'investing'
data = api.search_submissions(after=start_epoch,
                             subreddit = subreddit,
                             sort_type = 'score',
                             sort = 'desc',
                             filter=['author', 'title', 'subreddit' ,'score', 'selftext'],
                             limit=10000)


In [3]:
df = pd.DataFrame(list(data))
df = df[['created_utc','author','score','title','selftext']]
for i in range(len(df)):
    df['created_utc'].iloc[i] = time.strftime("%Y-%m-%d",time.gmtime(df['created_utc'].iloc[i]))

C:\Users\Gunther\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
df.columns=['Date','Name','Score','Title','Content']
df = df.set_index('Date')
df.head()

,Name,Score,Title,Content
Date,,,,
2020-01-03,ChocolateTsar,1510,Tesla tops Wall Street estimates with a record...,"&gt;Tesla delivered a record 112,000 vehicles ..."
2020-01-04,Fobulousguy,381,"Is there any good ELI5 resources on PUTS, SHOR...",
2020-01-02,FortyYearOldVirgin,380,"Dow jumps 250 points, hits record as Wall Stre...",[https://www.cnbc.com/2020/01/02/dow-futures-p...
2020-01-04,CIARRAPUNGI,340,What was your path in learning about the stock...,
2020-01-05,howtoreadspaghetti,312,I got bored and went through Amazon's financia...,[https://docs.google.com/spreadsheets/d/1naBI-...


In [6]:
df.to_csv('./Reddit Data/'+ subreddit + '_' +datetime.datetime.strftime(search_date,"%Y-%m-%d") + '.csv')

# Praw API
This is Reddit offical API can search specific word but less than 1000 data

In [ ]:
reddit = praw.Reddit(client_id='-RIN1YYCUr4Beg',
                     client_secret='2HifNYi9IFLXRDgGPh27IzbmMg8',
                     user_agent='crawler')

print(reddit.read_only)

In [ ]:
data = []

# Range 

subreddit = reddit.subreddit("investing")
keywords  = "fund"
limits    = 1000
time_filter = 'year'

# Import API
# https://praw.readthedocs.io/en/latest/code_overview/models/submission.html
# sort='relevance','hot','top','new','comments'
# syntax='lucene' , 'cloudsearch' , 'plain'
# time_filter='year'

for submission in subreddit.search(keywords, sort='relevance', syntax='lucene', time_filter=time_filter, limit=limits):
    date = time.gmtime(submission.created_utc)
    data.append([time.strftime("%Y-%m-%d",date),submission.author,submission.score,submission.title,submission.selftext])
    
df = pd.DataFrame(data,columns=['Date','Name','Score','Title','Content'])
df = df.set_index('Date')
df = df.dropna()
df.head()

In [ ]:
df.to_csv('./Reddit Data/keywords_'+ time_filter + '_'+ keywords +'.csv')

In [ ]:
data = []

# Range 
time_now = datetime.datetime.now()

subreddit = reddit.subreddit("investing")
keywords  = "hot"
limits    = 1000
time_filter = 'year'

for submission in subreddit.hot(limit=None):
    date = time.gmtime(submission.created_utc)
    data.append([time.strftime("%Y-%m-%d",date),submission.author,submission.score,submission.title,submission.selftext])
    
df = pd.DataFrame(data,columns=['Date','Name','Score','Title','Content'])
df = df.set_index('Date')
df

In [ ]:
df.to_csv('./Reddit Data/keywords_'+ time_filter + '_'+ keywords +'.csv')

# Push Shift API

The Json version for query data

In [ ]:
data = []

subreddit_name = 'investing'
days = '500d'

url = 'https://api.pushshift.io/reddit/submission/search/?after='+days+'&sort_type=created_utc&sort=asc&subreddit='+subreddit_name
df = pd.read_json(url,orient='split')
df

In [ ]:
dfa = df[['created_utc','author','score','title','selftext']]
dfa